# Análisis Exploratorio de E-Commerce con NumPy

**Objetivo:** En este notebook, realizaremos un análisis exploratorio de datos de un e-commerce del Reino Unido. El objetivo es demostrar cómo NumPy puede ser una herramienta poderosa para la **limpieza de datos**, **creación de métricas (feature engineering)** y **análisis estadístico segmentado**, todo ello con un rendimiento excepcional.

**Pasos:**
1.  **Carga y Preparación:** Convertiremos los datos de un DataFrame de Pandas a arrays de NumPy para el análisis numérico.
2.  **Limpieza de Datos:** Identificaremos y filtraremos registros problemáticos, como devoluciones o datos faltantes.
3.  **Análisis Descriptivo:** Calcularemos estadísticas fundamentales para entender el negocio.
4.  **Ejercicio Guiado:** Profundizaremos en un segmento específico de los datos.
5.  **Desafío para la Clase** ¡Les toca a ustedes!

In [1]:
import numpy as np
import pandas as pd

# --- Carga de Datos ---
# Cargamos el dataset desde el archivo CSV.
# Es importante especificar el encoding por los caracteres especiales que puede contener.
df = pd.read_csv('../../../datasets/alumnos/somosiete/E-CommerceAnalysusUK.csv', encoding='ISO-8859-1')


# Rellenamos los CustomerID faltantes con un valor placeholder (0) para poder convertir a numérico.
df['CustomerID'] = df['CustomerID'].fillna(0)
df.dropna(subset=['Description'], inplace=True) # Eliminamos filas donde la descripción es nula

# --- Conversión a NumPy ---
# Seleccionamos las columnas clave y las convertimos a un array de NumPy para el análisis.
# 1. Creamos una lista de tuplas a partir del DataFrame.
#    Acá definimos la "plantilla" de nuestro array.
#    Cada tupla representa una fila: (Quantity, UnitPrice, CustomerID)
registros = list(df[['Quantity', 'UnitPrice', 'CustomerID']].itertuples(index=False, name=None))

# 2. Definimos la estructura de nuestros datos (los "campos" o columnas)
#    'i4' es un entero de 4 bytes (int32), 'f8' es un float de 8 bytes (float64)
dtype_estructurado = [('Quantity', 'i4'), ('UnitPrice', 'f8'), ('CustomerID', 'i4')]

# 3. Creamos el array estructurado
datos_estructurados = np.array(registros, dtype=dtype_estructurado)

# Guardamos el país en un array separado para el análisis categórico.
paises = df['Country'].to_numpy(dtype=str)

print("Forma inicial del array de datos numéricos:", datos_estructurados.shape)
print("Forma inicial del array de países:", paises.shape)
print("\nPrimeras 5 filas de datos numéricos:\n", datos_estructurados[:5])

Forma inicial del array de datos numéricos: (540455,)
Forma inicial del array de países: (540455,)

Primeras 5 filas de datos numéricos:
 [(6, 2.55, 17850) (6, 3.39, 17850) (8, 2.75, 17850) (6, 3.39, 17850)
 (6, 3.39, 17850)]


### 2. Limpieza de Datos: La Base de un Buen Análisis

Los datos del mundo real rara vez son perfectos. Antes de cualquier análisis, es crucial limpiarlos. En este dataset, observamos dos problemas comunes:
*   **Cantidades (`Quantity`) negativas:** Probablemente representan devoluciones de productos.
*   **Precios (`UnitPrice`) de cero:** Podrían ser promociones, pero para un análisis de ingresos, distorsionan los resultados.

Usaremos el poder de la **indexación booleana** de NumPy para crear un "filtro" o "máscara" que seleccione únicamente las filas que cumplen con nuestros criterios de validez (cantidad > 0 y precio > 0).

In [12]:
# --- Creación de la Máscara de Limpieza ---
# 1. Condición para Cantidad: Accedemos al campo por su nombre.
cantidad_valida = datos_estructurados['Quantity'] > 0

# 2. Condición para Precio: Accedemos al campo 'UnitPrice'.
precio_valido = datos_estructurados['UnitPrice'] > 0

# 3. La combinación de máscaras funciona exactamente igual.
mascara_limpieza = cantidad_valida & precio_valido

# --- Aplicación de la Máscara ---
# Filtramos el array estructurado y el array de países.
datos_limpios = datos_estructurados[mascara_limpieza]
paises_limpios = paises[mascara_limpieza]

print(f"Filas antes de la limpieza: {datos_estructurados.shape[0]}")
print(f"Filas después de la limpieza: {datos_limpios.shape[0]}")
print(f"Se eliminaron {datos_estructurados.shape[0] - datos_limpios.shape[0]} filas.")

Filas antes de la limpieza: 540455
Filas después de la limpieza: 530104
Se eliminaron 10351 filas.


### 3. Feature Engineering: Creando la Métrica `IngresoTotal`

Ahora que tenemos datos limpios, podemos crear nuevas variables que nos den más información. La métrica más importante en un e-commerce es el ingreso.

Calcularemos el ingreso total por cada línea de producto (`Quantity` * `UnitPrice`). Esta es una demostración perfecta de la **vectorización** en NumPy: la operación se aplica a todos los elementos de los arrays de forma simultánea y ultra-rápida, sin necesidad de bucles.

In [ ]:
# --- Cálculo Vectorizado del Ingreso ---
# Extraemos las columnas del array estructurado limpio. 
cantidad = datos_limpios['Quantity']
precio_unitario = datos_limpios['UnitPrice']

# Multiplicación vectorizada de numpy
ingreso_por_linea = cantidad * precio_unitario


print("Cálculo de ingresos completado.")
print(f"Se generó un nuevo array de ingresos con forma: {ingreso_por_linea.shape}")
print("\nEjemplo de los primeros 5 ingresos calculados:\n")
print(np.round(ingreso_por_linea[:5], 2))

Cálculo de ingresos completado.
Se generó un nuevo array de ingresos con forma: (530104,)

Ejemplo de los primeros 5 ingresos calculados:

[15.3  20.34 22.   20.34 20.34]


### 4. Análisis Descriptivo: La Visión General del Negocio

Con nuestros datos limpios y enriquecidos, podemos calcular estadísticas agregadas para obtener una visión "macro" del comportamiento de las ventas. Utilizaremos las funciones de agregación optimizadas de NumPy.

In [16]:
# --- Cálculo de Estadísticas Agregadas ---
# Todas las funciones de agregación se aplican directamente sobre nuestro nuevo array.
ingreso_total_global = np.sum(ingreso_por_linea)
ingreso_promedio_por_linea = np.mean(ingreso_por_linea)
mediana_ingreso = np.median(ingreso_por_linea)
desv_est_ingreso = np.std(ingreso_por_linea)
transaccion_mas_valiosa = np.max(ingreso_por_linea)

print("--- Estadísticas Globales de Ingresos ---")
print(f"Ingreso Total Global: £{ingreso_total_global:,.2f}")
print(f"Ingreso Promedio por Línea de Producto: £{ingreso_promedio_por_linea:.2f}")
print(f"Mediana del Ingreso por Línea: £{mediana_ingreso:.2f}")
print(f"Desviación Estándar de los Ingresos: £{desv_est_ingreso:,.2f}")
print(f"Línea de Producto más Valiosa: £{transaccion_mas_valiosa:,.2f}")

--- Estadísticas Globales de Ingresos ---
Ingreso Total Global: £10,666,684.54
Ingreso Promedio por Línea de Producto: £20.12
Mediana del Ingreso por Línea: £9.90
Desviación Estándar de los Ingresos: £270.36
Línea de Producto más Valiosa: £168,469.60


### 5. Ejercicio Guiado: Análisis de Ventas por País

Una de las preguntas más comunes es comparar el rendimiento entre diferentes segmentos. Vamos a calcular el total de ingresos generados por un país específico, por ejemplo, **Alemania ('Germany')**, y ver qué porcentaje del total representa.

In [19]:
# 1. Crear la máscara para el país 'Germany' (sin cambios aquí).
mascara_alemania = (paises_limpios == 'Germany')

# 2. Aplicar la máscara directamente al array de ingresos.
ingresos_alemania = ingreso_por_linea[mascara_alemania]

# 3. El resto del cálculo es idéntico.
total_ingresos_alemania = np.sum(ingresos_alemania)
porcentaje_alemania = (total_ingresos_alemania / ingreso_total_global) * 100


print(f"Recordamos el ingreso Total Global: £{ingreso_total_global:,.2f}")
print(f"Total de Ingresos de Alemania: £{total_ingresos_alemania:,.2f}")
print(f"Alemania representa el {porcentaje_alemania:.2f}% del ingreso total.")

Recordamos el ingreso Total Global: £10,666,684.54
Total de Ingresos de Alemania: £228,867.14
Alemania representa el 2.15% del ingreso total.


### 6. 🚀 El Desafío: ¡Ahora les toca a ustedes!

Usando las mismas herramientas que ya vimos (máscaras, filtros, sumas), les toca responder la siguiente pregunta:

**¿Cuáles son los 5 países que más ingresos han generado para la compañía?**

**Pistas:**
1.  Consigan una lista de países únicos usando `np.unique(paises_limpios)`.
2.  Creen un bucle que itere sobre cada país.
3.  Dentro del bucle, creen una máscara booleana para el país actual (ej. `paises_limpios == pais`).
4.  Usen esa máscara para filtrar el array `ingreso_por_linea` y luego sumen los resultados.
5.  Guarden cada par en una lista y, al final, ordénala para encontrar el top 5.

¡Mucha suerte!